In [1]:
import numpy as np

print("Hello, World!")

Hello, World!


# Training for the TSN model

In [13]:
!python "C:\\Users\\rodol\\mmaction2\\tools\\train.py" "C:\\Users\\rodol\\mmaction2\\configs\\recognition\\tsn\\tsn_imagenet-pretrained-r50_8xb32-1x1x8-100e_kinetics400-rgb.py" --work-dir "C:\\Users\\rodol\\Desktop\\model_training_test\\tsn_test"

10/04 15:28:28 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: win32
    Python: 3.8.19 (default, Mar 20 2024, 19:55:45) [MSC v.1916 64 bit (AMD64)]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1316532037
    GPU 0: NVIDIA GeForce RTX 4070 Ti SUPER
    CUDA_HOME: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    MSVC: Microsoft (R) C/C++ Optimizing Compiler Version 19.38.33141 for x64
    GCC: n/a
    PyTorch: 2.0.1
    PyTorch compiling details: PyTorch built with:
  - C++ Version: 199711
  - MSVC 193431937
  - Intel(R) Math Kernel Library Version 2020.0.2 Product Build 20200624 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 2019
  - LAPACK is enabled (usually provided by MKL)
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - N

# Training for the I3D model

In [7]:
!python "C:\\Users\\rodol\\mmaction2\\tools\\train.py" "C:\\Users\\rodol\\mmaction2\\configs\\recognition\\i3d\\i3d_imagenet-pretrained-r50_8xb8-32x2x1-100e_kinetics400-rgb.py" --work-dir "C:\\Users\\rodol\\Desktop\\model_training_test"

10/02 23:42:02 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: win32
    Python: 3.8.19 (default, Mar 20 2024, 19:55:45) [MSC v.1916 64 bit (AMD64)]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 968628413
    GPU 0: NVIDIA GeForce RTX 4070 Ti SUPER
    CUDA_HOME: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    MSVC: Microsoft (R) C/C++ Optimizing Compiler Version 19.38.33141 for x64
    GCC: n/a
    PyTorch: 2.0.1
    PyTorch compiling details: PyTorch built with:
  - C++ Version: 199711
  - MSVC 193431937
  - Intel(R) Math Kernel Library Version 2020.0.2 Product Build 20200624 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 2019
  - LAPACK is enabled (usually provided by MKL)
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NV

# Training for the TIN model

In [8]:
# for tin
!python "C:\\Users\\rodol\\mmaction2\\tools\\train.py" "C:\\Users\\rodol\\mmaction2\\configs\\recognition\\tin\\tin_kinetics400-pretrained-tsm-r50_1x1x8-50e_kinetics400-rgb.py" --work-dir "C:\\Users\\rodol\\Desktop\\model_training_test\\tin_test"

^C


# Train for the Slowfast model (NOT COMPLETED YET)

In [ ]:
!python "C:\\Users\\rodol\\mmaction2\\tools\\train.py" "C:\\Users\\rodol\\mmaction2\\configs\\recognition\\slowfast\\slowfast_r50_8xb8-4x16x1-256e_kinetics400-rgb.py" --work-dir "C:\\Users\\rodol\\Desktop\\model_training_test\\slowfast_test"

# Inference

In [2]:
from mmaction.apis import inference_recognizer, init_recognizer


def basic_salsa_step_most_voted_inference(config_path, checkpoint_path, img_path):
    model = init_recognizer(config_path, checkpoint_path, device='cuda:0')
    summary = []
    class_tally_salsa_basic = 0
    class_tally_non_salsa_basic = 0
    for i in range(3):
        result = inference_recognizer(model, img_path)
        pred_scores = result.pred_score.tolist()
        # get the index of the highest score
        max_index = pred_scores.index(max(pred_scores))
        label_list = ["non_salsa_basic", "salsa_basic"]
        # get the label with the highest score
        label = label_list[max_index]
        summary.append({label: max(pred_scores)})
        if label == "salsa_basic":
            class_tally_salsa_basic += 1
        else:
            class_tally_non_salsa_basic += 1
    class_tally_summary = {"salsa_basic": class_tally_salsa_basic, "non_salsa_basic": class_tally_non_salsa_basic}
    # get class label that has the highest tally
    if class_tally_summary["salsa_basic"] > class_tally_summary["non_salsa_basic"]:
        print("final classification: salsa_basic")
    else:
        print("final classification: non_salsa_basic")
    return class_tally_summary

def basic_salsa_step_average_probability_for_inference(config_path, checkpoint_path, img_path):
    model = init_recognizer(config_path, checkpoint_path, device='cuda:0')
    summary = []
    class_tally_salsa_basic = 0
    class_tally_non_salsa_basic = 0
    for i in range(3):
        result = inference_recognizer(model, img_path)
        pred_scores = result.pred_score.tolist()
        max_index = pred_scores.index(max(pred_scores))
        label_list = ["non_salsa_basic", "salsa_basic"]
        label = label_list[max_index]
        summary.append({label: max(pred_scores)})
        if label == "salsa_basic":
            class_tally_salsa_basic += 1
        else:
            class_tally_non_salsa_basic += 1
   # for salsa_basic, get the average probability
    salsa_basic_sum = 0
    salsa_basic_count = 0
    for i in summary:
        if "salsa_basic" in i:
            salsa_basic_sum += i["salsa_basic"]
            salsa_basic_count += 1
    if salsa_basic_count == 0:
        salsa_basic_average = np.nan
    else:
        salsa_basic_average = salsa_basic_sum / salsa_basic_count
    # for non_salsa_basic, get the average probability
    non_salsa_basic_sum = 0
    non_salsa_basic_count = 0
    for i in summary:
        if "non_salsa_basic" in i:
            non_salsa_basic_sum += i["non_salsa_basic"]
            non_salsa_basic_count += 1
    if non_salsa_basic_count == 0:
        non_salsa_basic_average = np.nan
    else:
        non_salsa_basic_average = non_salsa_basic_sum / non_salsa_basic_count
    # get class label that has the highest average probability
    summary = {"salsa_basic": salsa_basic_average, "non_salsa_basic": non_salsa_basic_average}
    return summary

In [14]:
from mmaction.apis import inference_recognizer, init_recognizer

config_path_tsn = r"C:\Users\rodol\mmaction2\configs\recognition\tsn\tsn_imagenet-pretrained-r50_8xb32-1x1x8-100e_kinetics400-rgb.py"
checkpoint_path_tsn = r"C:\Users\rodol\Desktop\model_training_test\tsn_test\epoch_30.pth"

config_path_tin = r"C:\Users\rodol\mmaction2\configs\recognition\tin\tin_kinetics400-pretrained-tsm-r50_1x1x8-50e_kinetics400-rgb.py"
checkpoint_path_tin = r"C:\Users\rodol\Desktop\model_training_test\tin_test\epoch_35.pth"

config_path_i3d = r"C:\Users\rodol\mmaction2\configs\recognition\i3d\i3d_imagenet-pretrained-r50_8xb8-32x2x1-100e_kinetics400-rgb.py"
checkpoint_path_i3d = r"C:\Users\rodol\Desktop\model_training_test\i3d_test\epoch_15.pth"

config_path_slowfast = r"C:\Users\rodol\mmaction2\configs\recognition\slowfast\slowfast_r50_8xb8-4x16x1-256e_kinetics400-rgb.py"
checkpoint_path_slowfast = r"C:\Users\rodol\Desktop\model_training_test\slowfast_test\epoch_256.pth"


In [15]:
# salsa basic videos
img_path_salsa_basic_1 = r"C:\Users\rodol\Downloads\rudy_basic_step.mp4"
img_path_salsa_basic_2 = r"C:\Users\rodol\Desktop\E2E_Animation\mp4_videos\myle_salsa_dance_right_1.mp4"

# non salsa basic videos
img_path_non_salsa_basic_1 = r"C:\Users\rodol\Desktop\E2E_Animation\mp4_videos\non_salsa_basic\rando_move_1.mp4"
img_path_non_salsa_basic_2 = r"C:\Users\rodol\Desktop\E2E_Animation\mp4_videos\rando_salsa_dance_right_3.mp4"
img_path_non_salsa_basic_3 = r"C:\Users\rodol\Desktop\E2E_Animation\mp4_videos\non_salsa_basic\rando_move_2.mp4"
img_path_non_salsa_basic_4 = r"C:\Users\rodol\Desktop\E2E_Animation\mp4_videos\non_salsa_basic\rando_move_4.mp4"
img_path_non_salsa_basic_5 = r"C:\Users\rodol\Downloads\rudy_left_turn.mp4"
img_path_non_salsa_basic_6 = r"C:\Users\rodol\Downloads\rudy_right_turn.mp4"

In [5]:
model = init_recognizer(config_path_i3d, checkpoint_path_i3d, device='cuda:0')
result = inference_recognizer(model, img_path_salsa_basic_1)
pred_scores = result.pred_score.tolist()
print(pred_scores)

Loads checkpoint by local backend from path: C:\Users\rodol\Desktop\model_training_test\i3d_test\epoch_15.pth
10/04 15:24:35 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
10/04 15:24:35 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.
[0.004132361616939306, 0.9958677291870117]


In [12]:
model = init_recognizer(config_path_i3d, checkpoint_path_i3d, device='cuda:0')
result = inference_recognizer(model, img_path_non_salsa_basic_6)
pred_scores = result.pred_score.tolist()
print(pred_scores)

Loads checkpoint by local backend from path: C:\Users\rodol\Desktop\model_training_test\i3d_test\epoch_15.pth
[0.001334272907115519, 0.9986658096313477]


In [ ]:
model = init_recognizer(config_path_i3d, checkpoint_path_i3d, device='cuda:0')
result = inference_recognizer(model, img_path_salsa_basic_2)
pred_scores = result.pred_score.tolist()
print(pred_scores)

In [17]:
model = init_recognizer(config_path_tsn, checkpoint_path_tsn, device='cuda:0')
result = inference_recognizer(model, img_path_non_salsa_basic_6)
pred_scores = result.pred_score.tolist()
print(pred_scores)

Loads checkpoint by local backend from path: C:\Users\rodol\Desktop\model_training_test\tsn_test\epoch_30.pth
[3.890005245921202e-06, 0.9999960660934448]


In [61]:
tsn_most_voted_salsa_basic_1 = basic_salsa_step_most_voted_inference(config_path_tsn, checkpoint_path_tsn, img_path_salsa_basic_1)
tsn_average_probability_salsa_basic_1 = basic_salsa_step_average_probability_for_inference(config_path_tsn, checkpoint_path_tsn, img_path_salsa_basic_1)
tsn_most_voted_salsa_basic_2 = basic_salsa_step_most_voted_inference(config_path_tsn, checkpoint_path_tsn, img_path_salsa_basic_2)
tsn_average_probability_salsa_basic_2 = basic_salsa_step_average_probability_for_inference(config_path_tsn, checkpoint_path_tsn, img_path_salsa_basic_2)

tsn_most_voted_non_salsa_basic_1 = basic_salsa_step_most_voted_inference(config_path_tsn, checkpoint_path_tsn, img_path_non_salsa_basic_1)
tsn_average_probability_non_salsa_basic_1 = basic_salsa_step_average_probability_for_inference(config_path_tsn, checkpoint_path_tsn, img_path_non_salsa_basic_1)

tsn_most_voted_non_salsa_basic_2 = basic_salsa_step_most_voted_inference(config_path_tsn, checkpoint_path_tsn, img_path_non_salsa_basic_2)
tsn_average_probability_non_salsa_basic_2 = basic_salsa_step_average_probability_for_inference(config_path_tsn, checkpoint_path_tsn, img_path_non_salsa_basic_2)

tsn_most_voted_non_salsa_basic_3 = basic_salsa_step_most_voted_inference(config_path_tsn, checkpoint_path_tsn, img_path_non_salsa_basic_3)
tsn_average_probability_non_salsa_basic_3 = basic_salsa_step_average_probability_for_inference(config_path_tsn, checkpoint_path_tsn, img_path_non_salsa_basic_3)

tsn_most_voted_non_salsa_basic_4 = basic_salsa_step_most_voted_inference(config_path_tsn, checkpoint_path_tsn, img_path_non_salsa_basic_4)
tsn_average_probability_non_salsa_basic_4 = basic_salsa_step_average_probability_for_inference(config_path_tsn, checkpoint_path_tsn, img_path_non_salsa_basic_4)

tsn_most_voted_non_salsa_basic_5 = basic_salsa_step_most_voted_inference(config_path_tsn, checkpoint_path_tsn, img_path_non_salsa_basic_5)
tsn_average_probability_non_salsa_basic_5 = basic_salsa_step_average_probability_for_inference(config_path_tsn, checkpoint_path_tsn, img_path_non_salsa_basic_5)

tsn_most_voted_non_salsa_basic_6 = basic_salsa_step_most_voted_inference(config_path_tsn, checkpoint_path_tsn, img_path_non_salsa_basic_6)
tsn_average_probability_non_salsa_basic_6 = basic_salsa_step_average_probability_for_inference(config_path_tsn, checkpoint_path_tsn, img_path_non_salsa_basic_6)

Loads checkpoint by local backend from path: C:\Users\rodol\Desktop\model_training_test\tsn_test\epoch_30.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: backbone.conv.0.weight, backbone.conv.0.bias, backbone.conv.1.weight, backbone.conv.1.bias, backbone.conv.1.running_mean, backbone.conv.1.running_var, backbone.conv.1.num_batches_tracked, backbone.conv1.0.weight, backbone.conv1.0.bias, backbone.conv1.1.weight, backbone.conv1.1.bias, backbone.conv1.1.running_mean, backbone.conv1.1.running_var, backbone.conv1.1.num_batches_tracked, cls_head.fc.weight, cls_head.fc.bias

missing keys in source state_dict: backbone.conv1.conv.weight, backbone.conv1.bn.weight, backbone.conv1.bn.bias, backbone.conv1.bn.running_mean, backbone.conv1.bn.running_var, backbone.layer1.0.conv1.conv.weight, backbone.layer1.0.conv1.bn.weight, backbone.layer1.0.conv1.bn.bias, backbone.layer1.0.conv1.bn.running_mean, backbone.layer1.0.conv1.bn.running_var, backbone.layer1.

KeyboardInterrupt: 

In [55]:
tsn_most_voted_summary = [("salsa_basic_1", tsn_most_voted_salsa_basic_1), ("salsa_basic_2", tsn_most_voted_salsa_basic_2), ("non_salsa_basic_1", tsn_most_voted_non_salsa_basic_1), ("non_salsa_basic_2", tsn_most_voted_non_salsa_basic_2), ("non_salsa_basic_3", tsn_most_voted_non_salsa_basic_3), ("non_salsa_basic_4", tsn_most_voted_non_salsa_basic_4), ("non_salsa_basic_5", tsn_most_voted_non_salsa_basic_5), ("non_salsa_basic_6", tsn_most_voted_non_salsa_basic_6)]

# print each tuple in the list
print("TSN CONFIGURATION SUMMARY")
for i in tsn_most_voted_summary:
    print(i)

TSN CONFIGURATION SUMMARY
('salsa_basic_1', {'salsa_basic': 3, 'non_salsa_basic': 0})
('salsa_basic_2', {'salsa_basic': 0, 'non_salsa_basic': 3})
('non_salsa_basic_1', {'salsa_basic': 3, 'non_salsa_basic': 0})
('non_salsa_basic_2', {'salsa_basic': 3, 'non_salsa_basic': 0})
('non_salsa_basic_3', {'salsa_basic': 3, 'non_salsa_basic': 0})
('non_salsa_basic_4', {'salsa_basic': 0, 'non_salsa_basic': 3})
('non_salsa_basic_5', {'salsa_basic': 3, 'non_salsa_basic': 0})
('non_salsa_basic_6', {'salsa_basic': 3, 'non_salsa_basic': 0})


In [ ]:
tin_most_voted_salsa_basic_1 = basic_salsa_step_most_voted_inference(config_path_tin, checkpoint_path_tin, img_path_salsa_basic_1)
tin_average_probability_salsa_basic_1 = basic_salsa_step_average_probability_for_inference(config_path_tin, checkpoint_path_tin, img_path_salsa_basic_1)
tin_most_voted_salsa_basic_2 = basic_salsa_step_most_voted_inference(config_path_tin, checkpoint_path_tin, img_path_salsa_basic_2)
tin_average_probability_salsa_basic_2 = basic_salsa_step_average_probability_for_inference(config_path_tin, checkpoint_path_tin, img_path_salsa_basic_2)

tin_most_voted_non_salsa_basic_1 = basic_salsa_step_most_voted_inference(config_path_tin, checkpoint_path_tin, img_path_non_salsa_basic_1)
tin_average_probability_non_salsa_basic_1 = basic_salsa_step_average_probability_for_inference(config_path_tin, checkpoint_path_tin, img_path_non_salsa_basic_1)

tin_most_voted_non_salsa_basic_2 = basic_salsa_step_most_voted_inference(config_path_tin, checkpoint_path_tin, img_path_non_salsa_basic_2)
tin_average_probability_non_salsa_basic_2 = basic_salsa_step_average_probability_for_inference(config_path_tin, checkpoint_path_tin, img_path_non_salsa_basic_2)

tin_most_voted_non_salsa_basic_3 = basic_salsa_step_most_voted_inference(config_path_tin, checkpoint_path_tin, img_path_non_salsa_basic_3)
tin_average_probability_non_salsa_basic_3 = basic_salsa_step_average_probability_for_inference(config_path_tin, checkpoint_path_tin, img_path_non_salsa_basic_3)

tin_most_voted_non_salsa_basic_4 = basic_salsa_step_most_voted_inference(config_path_tin, checkpoint_path_tin, img_path_non_salsa_basic_4)
tin_average_probability_non_salsa_basic_4 = basic_salsa_step_average_probability_for_inference(config_path_tin, checkpoint_path_tin, img_path_non_salsa_basic_4)

tin_most_voted_non_salsa_basic_5 = basic_salsa_step_most_voted_inference(config_path_tin, checkpoint_path_tin, img_path_non_salsa_basic_5)
tin_average_probability_non_salsa_basic_5 = basic_salsa_step_average_probability_for_inference(config_path_tin, checkpoint_path_tin, img_path_non_salsa_basic_5)

tin_most_voted_non_salsa_basic_6 = basic_salsa_step_most_voted_inference(config_path_tin, checkpoint_path_tin, img_path_non_salsa_basic_6)
tin_average_probability_non_salsa_basic_6 = basic_salsa_step_average_probability_for_inference(config_path_tin, checkpoint_path_tin, img_path_non_salsa_basic_6)

In [58]:
tin_most_voted_summary = [("salsa_basic_1", tin_most_voted_salsa_basic_1), ("salsa_basic_2", tin_most_voted_salsa_basic_2), ("non_salsa_basic_1", tin_most_voted_non_salsa_basic_1), ("non_salsa_basic_2", tin_most_voted_non_salsa_basic_2), ("non_salsa_basic_3", tin_most_voted_non_salsa_basic_3), ("non_salsa_basic_4", tin_most_voted_non_salsa_basic_4), ("non_salsa_basic_5", tin_most_voted_non_salsa_basic_5), ("non_salsa_basic_6", tin_most_voted_non_salsa_basic_6)]

print("TIN CONFIGURATION SUMMARY")
# print each tuple in the list
for i in tin_most_voted_summary:
    print(i)

TIN CONFIGURATION SUMMARY
('salsa_basic_1', {'salsa_basic': 0, 'non_salsa_basic': 3})
('salsa_basic_2', {'salsa_basic': 0, 'non_salsa_basic': 3})
('non_salsa_basic_1', {'salsa_basic': 0, 'non_salsa_basic': 3})
('non_salsa_basic_2', {'salsa_basic': 0, 'non_salsa_basic': 3})
('non_salsa_basic_3', {'salsa_basic': 0, 'non_salsa_basic': 3})
('non_salsa_basic_4', {'salsa_basic': 0, 'non_salsa_basic': 3})
('non_salsa_basic_5', {'salsa_basic': 0, 'non_salsa_basic': 3})
('non_salsa_basic_6', {'salsa_basic': 0, 'non_salsa_basic': 3})


In [ ]:
i3d_most_voted_salsa_basic_1 = basic_salsa_step_most_voted_inference(config_path_i3d, checkpoint_path_i3d, img_path_salsa_basic_1)
i3d_average_probability_salsa_basic_1 = basic_salsa_step_average_probability_for_inference(config_path_i3d, checkpoint_path_i3d, img_path_salsa_basic_1)

i3d_most_voted_salsa_basic_2 = basic_salsa_step_most_voted_inference(config_path_i3d, checkpoint_path_i3d, img_path_salsa_basic_2)
i3d_average_probability_salsa_basic_2 = basic_salsa_step_average_probability_for_inference(config_path_i3d, checkpoint_path_i3d, img_path_salsa_basic_2)

i3d_most_voted_non_salsa_basic_1 = basic_salsa_step_most_voted_inference(config_path_i3d, checkpoint_path_i3d, img_path_non_salsa_basic_1)
i3d_average_probability_non_salsa_basic_1 = basic_salsa_step_average_probability_for_inference(config_path_i3d, checkpoint_path_i3d, img_path_non_salsa_basic_1)

i3d_most_voted_non_salsa_basic_2 = basic_salsa_step_most_voted_inference(config_path_i3d, checkpoint_path_i3d, img_path_non_salsa_basic_2)
i3d_average_probability_non_salsa_basic_2 = basic_salsa_step_average_probability_for_inference(config_path_i3d, checkpoint_path_i3d, img_path_non_salsa_basic_2)

i3d_most_voted_non_salsa_basic_3 = basic_salsa_step_most_voted_inference(config_path_i3d, checkpoint_path_i3d, img_path_non_salsa_basic_3)
i3d_average_probability_non_salsa_basic_3 = basic_salsa_step_average_probability_for_inference(config_path_i3d, checkpoint_path_i3d, img_path_non_salsa_basic_3)

i3d_most_voted_non_salsa_basic_4 = basic_salsa_step_most_voted_inference(config_path_i3d, checkpoint_path_i3d, img_path_non_salsa_basic_4)
i3d_average_probability_non_salsa_basic_4 = basic_salsa_step_average_probability_for_inference(config_path_i3d, checkpoint_path_i3d, img_path_non_salsa_basic_4)

i3d_most_voted_non_salsa_basic_5 = basic_salsa_step_most_voted_inference(config_path_i3d, checkpoint_path_i3d, img_path_non_salsa_basic_5)
i3d_average_probability_non_salsa_basic_5 = basic_salsa_step_average_probability_for_inference(config_path_i3d, checkpoint_path_i3d, img_path_non_salsa_basic_5)

i3d_most_voted_non_salsa_basic_6 = basic_salsa_step_most_voted_inference(config_path_i3d, checkpoint_path_i3d, img_path_non_salsa_basic_6)
i3d_average_probability_non_salsa_basic_6 = basic_salsa_step_average_probability_for_inference(config_path_i3d, checkpoint_path_i3d, img_path_non_salsa_basic_6)

In [57]:
i3d_most_voted_summary = [("salsa_basic_1", i3d_most_voted_salsa_basic_1), ("salsa_basic_2", i3d_most_voted_salsa_basic_2), ("non_salsa_basic_1", i3d_most_voted_non_salsa_basic_1), ("non_salsa_basic_2", i3d_most_voted_non_salsa_basic_2), ("non_salsa_basic_3", i3d_most_voted_non_salsa_basic_3), ("non_salsa_basic_4", i3d_most_voted_non_salsa_basic_4), ("non_salsa_basic_5", i3d_most_voted_non_salsa_basic_5), ("non_salsa_basic_6", i3d_most_voted_non_salsa_basic_6)]

print("I3D CONFIGURATION SUMMARY")
# print each tuple in the list
for i in i3d_most_voted_summary:
    print(i)

I3D CONFIGURATION SUMMARY
('salsa_basic_1', {'salsa_basic': 0, 'non_salsa_basic': 3})
('salsa_basic_2', {'salsa_basic': 0, 'non_salsa_basic': 3})
('non_salsa_basic_1', {'salsa_basic': 0, 'non_salsa_basic': 3})
('non_salsa_basic_2', {'salsa_basic': 0, 'non_salsa_basic': 3})
('non_salsa_basic_3', {'salsa_basic': 0, 'non_salsa_basic': 3})
('non_salsa_basic_4', {'salsa_basic': 3, 'non_salsa_basic': 0})
('non_salsa_basic_5', {'salsa_basic': 0, 'non_salsa_basic': 3})
('non_salsa_basic_6', {'salsa_basic': 0, 'non_salsa_basic': 3})


In [ ]:
slowfast_most_voted_salsa_basic_1 = basic_salsa_step_most_voted_inference(config_path_slowfast, checkpoint_path_slowfast, img_path_salsa_basic_1)
slowfast_average_probability_salsa_basic_1 = basic_salsa_step_average_probability_for_inference(config_path_slowfast, checkpoint_path_slowfast, img_path_salsa_basic_1)

slowfast_most_voted_salsa_basic_2 = basic_salsa_step_most_voted_inference(config_path_slowfast, checkpoint_path_slowfast, img_path_salsa_basic_2)
slowfast_average_probability_salsa_basic_2 = basic_salsa_step_average_probability_for_inference(config_path_slowfast, checkpoint_path_slowfast, img_path_salsa_basic_2)

slowfast_most_voted_non_salsa_basic_1 = basic_salsa_step_most_voted_inference(config_path_slowfast, checkpoint_path_slowfast, img_path_non_salsa_basic_1)
slowfast_average_probability_non_salsa_basic_1 = basic_salsa_step_average_probability_for_inference(config_path_slowfast, checkpoint_path_slowfast, img_path_non_salsa_basic_1)

slowfast_most_voted_non_salsa_basic_2 = basic_salsa_step_most_voted_inference(config_path_slowfast, checkpoint_path_slowfast, img_path_non_salsa_basic_2)
slowfast_average_probability_non_salsa_basic_2 = basic_salsa_step_average_probability_for_inference(config_path_slowfast, checkpoint_path_slowfast, img_path_non_salsa_basic_2)

slowfast_most_voted_non_salsa_basic_3 = basic_salsa_step_most_voted_inference(config_path_slowfast, checkpoint_path_slowfast, img_path_non_salsa_basic_3)
slowfast_average_probability_non_salsa_basic_3 = basic_salsa_step_average_probability_for_inference(config_path_slowfast, checkpoint_path_slowfast, img_path_non_salsa_basic_3)

slowfast_most_voted_non_salsa_basic_4 = basic_salsa_step_most_voted_inference(config_path_slowfast, checkpoint_path_slowfast, img_path_non_salsa_basic_4)
slowfast_average_probability_non_salsa_basic_4 = basic_salsa_step_average_probability_for_inference(config_path_slowfast, checkpoint_path_slowfast, img_path_non_salsa_basic_4)

slowfast_most_voted_non_salsa_basic_5 = basic_salsa_step_most_voted_inference(config_path_slowfast, checkpoint_path_slowfast, img_path_non_salsa_basic_5)
slowfast_average_probability_non_salsa_basic_5 = basic_salsa_step_average_probability_for_inference(config_path_slowfast, checkpoint_path_slowfast, img_path_non_salsa_basic_5)

slowfast_most_voted_non_salsa_basic_6 = basic_salsa_step_most_voted_inference(config_path_slowfast, checkpoint_path_slowfast, img_path_non_salsa_basic_6)
slowfast_average_probability_non_salsa_basic_6 = basic_salsa_step_average_probability_for_inference(config_path_slowfast, checkpoint_path_slowfast, img_path_non_salsa_basic_6)

In [59]:
slowfast_most_voted_summary = [("salsa_basic_1", slowfast_most_voted_salsa_basic_1), ("salsa_basic_2", slowfast_most_voted_salsa_basic_2), ("non_salsa_basic_1", slowfast_most_voted_non_salsa_basic_1), ("non_salsa_basic_2", slowfast_most_voted_non_salsa_basic_2), ("non_salsa_basic_3", slowfast_most_voted_non_salsa_basic_3), ("non_salsa_basic_4", slowfast_most_voted_non_salsa_basic_4), ("non_salsa_basic_5", slowfast_most_voted_non_salsa_basic_5), ("non_salsa_basic_6", slowfast_most_voted_non_salsa_basic_6)]


print("SLOWFAST CONFIGURATION SUMMARY")
# print each tuple in the list
for i in slowfast_most_voted_summary:
    print(i)

SLOWFAST CONFIGURATION SUMMARY
('salsa_basic_1', {'salsa_basic': 3, 'non_salsa_basic': 0})
('salsa_basic_2', {'salsa_basic': 0, 'non_salsa_basic': 3})
('non_salsa_basic_1', {'salsa_basic': 0, 'non_salsa_basic': 3})
('non_salsa_basic_2', {'salsa_basic': 0, 'non_salsa_basic': 3})
('non_salsa_basic_3', {'salsa_basic': 0, 'non_salsa_basic': 3})
('non_salsa_basic_4', {'salsa_basic': 0, 'non_salsa_basic': 3})
('non_salsa_basic_5', {'salsa_basic': 3, 'non_salsa_basic': 0})
('non_salsa_basic_6', {'salsa_basic': 3, 'non_salsa_basic': 0})


In [62]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:41:10_Pacific_Daylight_Time_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [65]:
!gcc -V

'gcc' is not recognized as an internal or external command,
operable program or batch file.


In [64]:
!nvidia-smi

Fri Sep 20 10:08:36 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.81                 Driver Version: 560.81         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   32C    P8             10W /  285W |    9136MiB /  16376MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [71]:
# Check Pytorch installation
import torch, torchvision
print("Torch version is: "+torch.__version__, "Is cuda device available? "+str(torch.cuda.is_available()))

# Check MMAction2 installation
import mmaction
print("mmaction version is: "+mmaction.__version__)

# Check MMCV installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print("MMCV version is: "+str(get_compiling_cuda_version()))
print(get_compiler_version())

# Check cuda version
!nvcc -V

# Check MMEngine installation
# from mmengine.utils.dl_utils import collect_env
# print(collect_env())

Torch version is: 2.0.1 Is cuda device available? True
mmaction version is: 1.2.0
MMCV version is: 11.8
MSVC 192829924
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:41:10_Pacific_Daylight_Time_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
